In [381]:
import numpy as np

In [382]:
def load_tsp_file(file_path):
    tsp_data = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Flag to indicate the start of the NODE_COORD_SECTION
    start_loading = False

    for line in lines:
        # Strip leading and trailing whitespaces
        line = line.strip()

        # Check for the start of the NODE_COORD_SECTION
        if line == "NODE_COORD_SECTION":
            start_loading = True
            continue

        # Check for the end of the file
        if line == "EOF":
            break

        if start_loading:
            parts = line.split()
            
            # Extract node number, x-coordinate, and y-coordinate
            node_number = int(parts[0])
            x_coordinate = float(parts[1])
            y_coordinate = float(parts[2])

            # Store data in the dictionary
            tsp_data[node_number] = np.array([x_coordinate, y_coordinate])

    # Check if the dataset is as intuitive as it appears
    nodes = set(tsp_data.keys())
    highest_number = max(nodes)
    missing_nodes = set(range(1, highest_number + 1)) - nodes

    if missing_nodes:
        print(f"Note that not all numbers between 0 and {highest_number} are used as a node ID. Node(s) {missing_nodes} are missing.")

    return tsp_data

In [383]:
def generate_solution(tsp_data):
    # Make random order as a starting solution
    return np.random.permutation(list(tsp_data.keys()))

In [384]:
def get_score(tsp_data, solution):
    # Calculate distance of a solution
    distance = 0
    coordinates = solution[0]
    for city in solution[1:]:
        next_coordinates = tsp_data[city]
        distance += np.linalg.norm(coordinates - next_coordinates)
        coordinates = next_coordinates.copy()

    return distance

In [385]:
def optimize(start_solution, tsp_data, iterations, mutate, accept, parameters):
    """
    Optimize solution for traveling salesman problem according to chosen mutate and accept function
    """ 

    solution = start_solution[:]
    current_score = get_score(tsp_data, solution)
    
    # Do iterations
    for i in range(iterations):

        # Make new solution via chosen mutation function
        new_solution = mutate(solution)
        new_score = get_score(tsp_data, new_solution)

        # Accept or deny new solution according to chosen acceptance function and corresponding parameters
        if accept(current_score, new_score, parameters):
            solution = new_solution[:]
            current_score = new_score

    return solution, current_score

In [386]:
def swap(solution):
    # Randomly choose two distinct indices
    indices = np.random.choice(len(solution), size=2, replace=False)

    new_solution = solution[:]

    # Swap the elements at the selected indices
    new_solution[indices[0]], new_solution[indices[1]] = new_solution[indices[1]], new_solution[indices[0]]

    return new_solution

In [387]:
def hillclimber(current_score, new_score, parameters):
    # Hillclimber function, accept only better scores
    return current_score > new_score


In [388]:
file = 'eil51'

file_path = f'TSP-Configurations/{file}.tsp.txt'
tsp_data = load_tsp_file(file_path)

start_solution = generate_solution(tsp_data)

iterations = 2000
mutate = swap
accept = hillclimber
parameters = []
best_solution, score = optimize(start_solution, tsp_data, iterations, mutate, accept, parameters)

print(score)


1368.3276706945123
